In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import json

from ngsolve import *

from ngsolve.meshes import *
#from ngsolve.solvers import *
from netgen.geom2d import unit_square


#from su2_yangmills import *
from lgt import *

from ngsolve.webgui import Draw

In [ ]:
from pathlib import Path
from ngsolve.fem import CompilePythonModule

lgtcpp_path = "../../src/gaugelinkfe/lgt.cpp" 

if True:
    print("compile it")
    m = CompilePythonModule(Path(lgtcpp_path), init_function_name="lgt_module")
else:
    print("load it")
    #import lgt_module as m

In [ ]:
parameters = {}
#parameters["h"] = 0.05
#parameters["h"] = 0.02
#parameters["h"] = 0.01
parameters["h"] = 0.1
parameters["n"] = int(1/parameters["h"])
#parameters["n"] = 2
parameters["order"] = 4
#parameters["dt"] = 0.001
parameters["dt"] = 0.0001
#parameters["dt"] = 0.00005

In [ ]:
ne=parameters["n"]
#mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=True, periodic_y=True)
mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#mesh = MakeStructured2DMesh(quads=False, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw(mesh)
meshvars = m.MeshVars(mesh)

In [ ]:
#check_facet_vorb(meshvars)
#draw_tbone_weights(meshvars)
#draw_bone_weights(meshvars)
#check_ellink_orientation(meshvars)

In [ ]:
order=parameters["order"]
# for su2 charge
#fesC = L2(mesh, order=order, dgjumps=True)**3

# TODO: unable to access components of gridfunction - gives index error
#fesC = Periodic(L2(mesh, order=order)**3)
#fesG = Periodic(L2(mesh, order=0)**4)

# Lie algebra space
fesC = L2(mesh, order=order)**3

# quaternion space!
fesG = L2(mesh, order=0)**4

# upwind flux of current through faces
#fesjflux = FacetFESpace(mesh, order=0)**3


# parallel transport maps between neighbouring volumes, belonging to an oriented interface
#TODO: orientation is handled by a special CF
# quaternion space!
#fesU = FacetFESpace(mesh, order=0, dgjumps=True, dirichlet=".*")**4
fesU = FacetFESpace(mesh, order=0, dgjumps=True)**4



# define a global orientation of links
fesHd = HDiv(mesh, order=0)
gfor = GridFunction(fesHd)
#gfor = GridFunction(HDiv(mesh, order=0))
gfor.vec.data[:] = 1.
n = specialcf.normal(mesh.dim)



# space of wilson loops centered around bones (= corners (2D) or edges (3D))
# for visualization only!
fesW = H1(mesh, order=order)**4

# space of action integrand

fesS = H1(mesh, order=order)

In [ ]:
def gaussxy(mu, sigma2):
    return gaussx(mu[0], sigma2) * gaussy(mu[1], sigma2)

def gaussx(mu, sigma2):
    return 1./sqrt(2*pi*sigma2) * exp(-0.5/sigma2 *( (x-mu)*(x-mu)))

def gaussy(mu, sigma2):
    return 1./sqrt(2*pi*sigma2) * exp(-0.5/sigma2 *( (y-mu)*(y-mu)))

In [ ]:
# set the scenarios

# wind:

def rot_wind():
    return CF( (y-0.5,-(x-0.5)) ) 

def x_wind():
    return CF( (10,0) )

def y_wind():
    return CF( (0,1) )

def my_wind():
    return CF( (0,-1) )

def mxmy_wind():
    return CF( (-1,-1) )


#rot_wind()
#tend = 5.00 # numerical dispersion and oscillations kick in
tend_rot_wind = 6.3 # one rotation
#tend = 3.


#mxmy_wind()
#tend = 5.00 
tend_mxmy_wind = 0.3


#???
#tend_x_wind = 0.2
tend_x_wind = 0.4

# charge distribution:

mu = [0.5,0.3]
mu_l = [0.3,0.5]
mu_c = [0.5,0.5]
#sigma2 = 0.01
sigma2 = 0.002

def rho_center(gfrho):
    gfrho.Set( CF( (gaussxy(mu_c, sigma2), 0, 0) ) )
    return gfrho

def rho_d(gfrho):
    gfrho.Set( CF( (gaussxy(mu, sigma2), 0, 0) ) )
    return gfrho

def rho_l(gfrho):
    gfrho.Set( CF( (0.01*gaussxy(mu_l, sigma2), 0, 0) ) )
    return gfrho

def zero(gfrho):
    gfrho.Set( CF( (0, 0, 0) ) )
    return gfrho

def red(gfrho):
    gfrho.Set( CF( (1, 0, 0) ) )
    return gfrho


# background gauge links:
e_num=220
q_theta = 0.1*pi
theta =0.
phi=0.


#d
el_num = 88

#r
el_num = 177
el_num = 193


scenario = {}

#scenario["wind"] =  rot_wind
scenario["wind"] =  x_wind
#scenario["wind"] =  mxmy_wind

#scenario["rho"] = rho_d
scenario["rho"] = rho_center
#scenario["rho"] = rho_l
#scenario["rho"] = red
scenario["rho"] = zero

#scenario["tend"] = tend_rot_wind
scenario["tend"] = tend_x_wind

scenario["gfU"] = trivial_gauge
#scenario["gfU"] = lambda _U: single_link(_U, mesh, e_num, qtheta=0.25*pi, theta=0., phi=0.)
#scenario["gfU"] = lambda _U: single_link(_U, mesh, 40, qtheta=0.01*pi, theta=0., phi=0.)
#scenario["gfU"] = lambda _U: single_link(_U, mesh, 150, qtheta=0.0001*pi, theta=0., phi=0.)
#scenario["gfU"] = lambda _U: single_link(_U, mesh, 208, qtheta=0.0001*pi, theta=0., phi=0.)


scenario["gauge"] = trivial_gauge
#scenario["gauge"] = lambda _gfg: single_el_gauge(_gfg, el_num, qtheta=0.25*pi, theta=0., phi=0.)

#scenario["name"] = "diag_singleU"
#scenario["name"] = "cw_singleU"
scenario["name"] = "x_trivial"

In [ ]:
# set wind
cfwind = scenario["wind"]()

hcwind = GridFunction( HCurl(mesh,order=1) )
hcwind.Set(cfwind)
Draw(cfwind, mesh)

In [ ]:
#set gauge links

gfU = GridFunction(fesU)

#set_gfU_const(gfU, qtheta=0.1*pi, theta = 0., phi=0.)
#set_gfU_link(gfU, enum=220, qtheta=0.1*pi, theta = 0., phi=0.)

#gfU = scenario["gfU"](gfU)
scenario["gfU"](gfU)

#random_links(gfU, 0.1*pi, np.pi, 2*np.pi)

gfS = GridFunction(fesS)
m.CalcgfHB(meshvars,gfU,gfS)
print(f" Norm(gfS)**2 = {Integrate((gfS)*(gfS), mesh)}")

# the orientation of gauge links coincides with that of facet normal vectors!
# the gauge links need to take this into account 
# convention: gfU links go along global normal vector
# gfUglob links point out of the element
# TODO: ist this consistent with C++ code, where orientation is based on vertex numbering?
gfUglob = IfPos(InnerProduct(gfor,n), gfU, qconjCF(gfU))



# for the current coupling term J*A at facets
# the "gauge fields" direction needs to be taken into account

#gfA = GridFunction(fesA)

#gfA.Set( (qlogCF(gfU))[1:] )

# similar orientation issue with gfU/gfUglob
#gfAglob = IfPos(gfor*n, gfA, -1.*gfA)

In [ ]:
# set charge density

gfrho = GridFunction(fesC)

gfrho = scenario["rho"](gfrho)

#Draw(Norm(gfrho), mesh)
#Draw(gfrho.components[0], mesh)
Draw(gfrho, mesh)
#print(Integrate(Norm(gfrho), mesh))

In [ ]:
# apply gauge transformation
gfg = GridFunction(fesG)

gfg = scenario["gauge"](gfg)
#scenario["gauge"](gfg)
#Draw(gfg.components[0], mesh)

print("before gauging")
Draw(gfrho, mesh)
gauge_rho(gfrho, gfg)
print("after gauging")
Draw(gfrho, mesh)


print("before gauging")


print("after gauging")

gauge_gfU(gfU, gfg, fesU)

In [ ]:
# caclulate integrated charge per element
gfrhoint = GridFunction(L2(mesh, order=0)**3)

gfrhoint_vecs = []

# list of
# ngsolve.bla.VectorD?
# for each color component
gaussfluxint = []


# calculate integrated charge per element
for c in range(3):
    gfrhoint_vecs.append( Integrate(gfrho.components[c], mesh, element_wise=True) )
    print(type(gfrhoint_vecs[-1]))
    print(len(gfrhoint.components[c].vec.data))
    print(len(gfrhoint_vecs[c]))
    gfrhoint.components[c].vec.data[:] = np.asarray(gfrhoint_vecs[c][:])
    #gfrhoint.components[c].vec.data[:] = gfrhoint_vecs[c][:]

In [ ]:
gfrhoint.components[0].vec.data[:] = 0.

In [ ]:
#random_links(gfU, 0.1*pi, np.pi, 2*np.pi)

enums = [27, 100, 150, 157, 160]
qU = get_rotq(0.1*pi, [1., 0., 0.])
qthetas = [0.1*pi for i in range(len(enums))]
thetas = [0 for i in range(len(enums))]
phis = [0 for i in range(len(enums))]
glob_ors = [True for i in range(len(enums))]
#set_gfU_links(gfU, enums, qthetas, thetas, phis, glob_ors)

gfUnew = GridFunction(fesU)
gfUnew.vec.data[:] = gfU.vec.data[:] 

# perturb one link:

#enum = 27

#diff_enums = [27, 100, 150, 157, 160]
diff_enums = [208]
#diff_enums = []

qrot = get_rotq(0.005*pi, [1., 0., 0.])

#draw_edge(enum, mesh)
print("Edges:")
draw_edges(enums, mesh)
#for enum in diff_enums:
for enum in enums:
    if(meshvars.FacetVorb(enum) == False):
        raise ValueError(f"edge number {enum} is not a volume element!")
    
    e_mesh = mesh[NodeId(EDGE,enum)]
    print(e_mesh.faces)
    el_num = e_mesh.faces[0].nr
    print(el_num)
    #gfU.components[0].vec.data[enum] = 0.
    #gfUnew.components[0].vec.data[enum] = 0.
    
    Ulink = np.array(get_qlink(gfUnew, e_mesh))
    print(Ulink)
    
    #qrot = get_rotq(0.*pi, [1., 0., 0.])
    Ulinkrot = qmul(qrot, Ulink)
    print(Ulinkrot)

    # LINKS FOR NEW FIELD
    set_gfU_link_q(gfUnew, enum, Ulinkrot)
    # LINKS FOR OLD FIELD
    #set_gfU_link_q(gfU, enum, Ulinkrot)
    
#set_gfU_link(gfUnew, enum, qtheta=0., theta=0., phi=0., glob_or=True)

eps = 0.000000001


# checking gauss constraint - should be only violated in pertubation region



print("gfHB gfU")
gfS = GridFunction(fesS)
m.CalcgfHB(meshvars,gfU,gfS)
#print(f" Norm(gfS)**2 = {Integrate((gfS)*(gfS), mesh)}")
gfSscene = Draw(gfS, mesh)

print("gfHB gfUnew")
gfSnew = GridFunction(fesS)
m.CalcgfHB(meshvars,gfUnew,gfSnew)
gfSnewscene = Draw(gfSnew, mesh)

fesl2 = L2(mesh, order=0)
gfgausserr2 = GridFunction(fesl2)
#gfgausserr2.vec.data[:] = 0.
print("gfgausserr2")
err2scene = Draw(gfgausserr2, mesh)

def gfl2_gausserr2(ne, eps, gfgausserr2, scene):
    for elnum in range(ne*ne):
        #print ("\r", f"elnum={elnum}", end="")
        gfgausserr2.vec.data[elnum] = m.GaussElementCheck(meshvars, gfU, gfUnew, gfrhoint, elnum, eps)
        #time.sleep(1)
    scene.Redraw()

gfl2_gausserr2(ne, eps, gfgausserr2, err2scene)
#err2scene.Redraw()
#err2scene = Draw(gfgausserr2, mesh)

In [ ]:
gfS = GridFunction(fesS)
m.CalcgfHB(meshvars,gfU,gfS)
#print(f" Norm(gfS)**2 = {Integrate((gfS)*(gfS), mesh)}")
gfSscene = Draw(gfS, mesh)

gfSnew = GridFunction(fesS)
m.CalcgfHB(meshvars,gfUnew,gfSnew)
gfSnewscene = Draw(gfSnew, mesh)

fesl2 = L2(mesh, order=0)
gfgausserr2 = GridFunction(fesl2)
#gfgausserr2.vec.data[:] = 0.
err2scene = Draw(gfgausserr2, mesh)
gfl2_gausserr2(ne, eps, gfgausserr2, err2scene)
err2scene.Redraw()

In [ ]:
print("Start update loop")
input()
el_gauss_err2 = []
gauss_err2 = []
S_list = []
S_dev_list = []


eps = 0.001

maxit = 1000
for t in range(maxit):
    el_gauss_err2.append( m.GaussElementCheck(meshvars, gfU, gfUnew, gfrhoint, el_num, eps) )
    gauss_err2.append( m.GaussCheck(meshvars, gfU, gfUnew, gfrhoint, eps) )
    m.GaussUpdate(meshvars, gfU, gfUnew, gfrhoint, eps)
    m.CalcgfHB(meshvars,gfUnew,gfSnew)
    gfSscene.Redraw()
    gfSnewscene.Redraw()
    gfl2_gausserr2(ne, eps, gfgausserr2, err2scene)
    
    S_dev_list.append(Integrate((gfS - gfSnew)*(gfS - gfSnew), mesh))
    S_list.append(Integrate(gfS*gfS, mesh))
    #time.sleep(0.001)
    print ("\r", f"iteration : {t}", end="")
    #input()






In [ ]:
print(el_gauss_err2[-1])
print(m.GaussElementCheck(meshvars, gfU, gfUnew, gfrhoint, el_num, eps))
#print(el_gauss_err2)
plt.plot( range(maxit), np.log(el_gauss_err2) )
plt.plot( range(maxit), np.log(gauss_err2) )
#plt.plot( range(maxit), S_list )
#plt.plot( range(maxit), S_dev_list )

In [ ]:
#Draw(gfS, mesh)
Draw(gfSnew - gfS, mesh)